In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import json

In [2]:
from konlpy.tag import Okt
okt = Okt()

In [3]:
openFileName = '전자담배_naver_news_1000.json'
    
f = open(openFileName, 'r', encoding='utf-8').read()

jsonData = json.loads(f)
data = ''

for item in jsonData:
    if 'description' in item.keys():
        data = data + re.sub(r'[^\w]', ' ', item['description']) + ' '
        
# f.close()

In [4]:
def tokenizer(raw, pos=["Noun"], stopword=[]): # 다른 품사 추가 원할 시 pos=["Noun", "Josa", ...]
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

documents1 = tokenizer(data)

In [5]:
import gensim

documents2 = [documents1] # gensim.corpora.Dictionary() 는 매개변수로 리스트 타입을 받기 때문에
dictionary = gensim.corpora.Dictionary(documents2)
print(dictionary.token2id)
print('dictionary size : %d' % len(dictionary)) # dictionary size : 37987

{'가격': 0, '가까이': 1, '가능': 2, '가능성': 3, '가담': 4, '가득': 5, '가래침': 6, '가량': 7, '가로': 8, '가맹': 9, '가수': 10, '가스': 11, '가시': 12, '가연성': 13, '가열': 14, '가운데': 15, '가의': 16, '가이드라인': 17, '가인': 18, '가장': 19, '가전': 20, '가전제품': 21, '가정': 22, '가족': 23, '가주': 24, '가지': 25, '가치': 26, '가파르': 27, '가해': 28, '가해자': 29, '가향': 30, '가혹': 31, '각각': 32, '각광': 33, '각국': 34, '각종': 35, '간의': 36, '간이': 37, '간접': 38, '간접흡연': 39, '간주': 40, '간판': 41, '갈무리': 42, '갈수록': 43, '감격': 44, '감고': 45, '감금': 46, '감기': 47, '감독': 48, '감동': 49, '감사원': 50, '감소': 51, '감안': 52, '감지': 53, '갑자기': 54, '강남': 55, '강남구': 56, '강덕': 57, '강도': 58, '강력': 59, '강민우': 60, '강세': 61, '강신우': 62, '강요': 63, '강우': 64, '강조': 65, '강주현': 66, '강화': 67, '개국': 68, '개꼴': 69, '개념': 70, '개량': 71, '개막': 72, '개발': 73, '개발도상국': 74, '개비': 75, '개선': 76, '개시': 77, '개월': 78, '개인': 79, '개정': 80, '개정안': 81, '개척': 82, '객수': 83, '객실': 84, '갤러리': 85, '거기': 86, '거나': 87, '거래': 88, '거래소': 89, '거리': 90, '거부': 91, '거울': 92, '거절': 93, '거짓말': 94, '거처': 95, '거치': 96, '걱정': 97, 

In [6]:
corpus_path = '전자담배_naver_news_1000.json'

class Documents:
    def __init__(self, path):
        self.path = path
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield doc.strip().split()

documents = Documents(corpus_path)

In [7]:
class Corpus:
    def __init__(self, path, dictionary):
        self.path = path
        self.dictionary = dictionary
        self.length = 0
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield self.dictionary.doc2bow(doc.split())
    def __len__(self):
        if self.length == 0:
            with open(self.path, encoding='utf-8') as f:
                for i, doc in enumerate(f):
                    continue
            self.length = i + 1
        return self.length

corpus = Corpus(corpus_path, dictionary)
for i, doc in enumerate(corpus):
    #if i >= 5: break
    print(doc)

[]
[]
[(204, 2), (334, 1), (1348, 1), (1935, 1), (1997, 1), (2317, 1)]
[]
[]
[]
[(744, 1), (1997, 1), (2035, 1)]
[]
[]
[(1771, 1)]
[]
[]
[]
[(375, 1)]
[]
[]
[(174, 1), (279, 1), (708, 1), (1311, 1), (1530, 1)]
[]
[]
[]
[]
[]
[]
[(421, 1), (464, 1), (713, 1), (2145, 1), (2182, 1), (2391, 1)]
[]
[]
[]
[]
[]
[]
[(376, 1), (1017, 1)]
[]
[]
[]
[(204, 1), (744, 1)]
[]
[]
[(260, 1), (427, 1), (1539, 1), (1602, 1), (1896, 1)]
[]
[]
[]
[(1765, 1), (1896, 1), (2259, 1)]
[]
[]
[(626, 1)]
[]
[]
[]
[]
[]
[]
[(204, 1), (744, 1), (1997, 1)]
[]
[]
[]
[(204, 1), (744, 1), (1997, 1)]
[]
[]
[(204, 2), (334, 1), (1348, 1), (1935, 1), (1997, 1), (2317, 1)]
[]
[]
[]
[(204, 1), (1714, 1), (1997, 1)]
[]
[]
[(204, 1), (1016, 1), (1997, 1)]
[]
[]
[]
[(204, 1), (744, 1), (1997, 1)]
[]
[]
[(204, 1), (376, 1), (1017, 1), (1997, 1)]
[]
[]
[]
[(204, 1), (1997, 1)]
[]
[]
[(376, 1), (1017, 1)]
[]
[]
[]
[(1211, 1), (2242, 1)]
[]
[]
[(174, 1), (279, 1), (626, 1), (708, 1), (1226, 1), (1227, 1), (1311, 1), (2050, 1)]
[]


[]
[]
[(476, 1), (2030, 1), (2265, 1)]
[]
[]
[(238, 1), (521, 1), (626, 1), (2294, 1)]
[]
[]
[]
[]
[]
[]
[(300, 1), (349, 1), (994, 1), (1979, 1)]
[]
[]
[]
[]
[]
[]
[(19, 1), (238, 1), (1123, 1), (1226, 1), (2003, 1), (2271, 1)]
[]
[]
[]
[]
[]
[]
[(260, 2), (466, 1), (521, 1), (909, 2), (1179, 1), (1342, 1), (1709, 1), (2034, 1), (2145, 1), (2271, 1)]
[]
[]
[]
[(1181, 1)]
[]
[]
[(32, 1), (238, 1), (626, 1), (656, 1), (1402, 1)]
[]
[]
[]
[(466, 1)]
[]
[]
[(260, 1), (805, 1), (1055, 1), (1179, 1), (2174, 1), (2176, 1), (2294, 2)]
[]
[]
[]
[]
[]
[]
[(296, 1), (423, 1), (626, 1), (1179, 1)]
[]
[]
[]
[]
[]
[]
[(398, 1), (417, 1), (485, 1), (488, 1), (617, 1), (2003, 1)]
[]
[]
[]
[]
[]
[]
[(330, 1), (888, 1), (1123, 1), (1226, 1), (2347, 1), (2390, 1)]
[]
[]
[]
[]
[]
[]
[(466, 1), (521, 1), (888, 1), (1123, 1), (1226, 1), (1402, 1), (1662, 1), (2347, 1), (2390, 1)]
[]
[]
[]
[]
[]
[]
[(238, 2), (713, 1), (1402, 1), (2003, 1)]
[]
[]
[]
[(1179, 1), (2034, 1)]
[]
[]
[(19, 1), (300, 1), (1902, 1)

[]
[]
[]
[]
[]
[(488, 1), (950, 1), (1015, 1), (1078, 1), (1357, 1), (1392, 1), (1408, 1)]
[]
[]
[]
[(644, 1)]
[]
[]
[(260, 1), (713, 1), (1596, 1)]
[]
[]
[]
[(2174, 1), (2242, 1)]
[]
[]
[(490, 1), (713, 2), (852, 1)]
[]
[]
[]
[(1190, 1)]
[]
[]
[(24, 2), (1596, 1)]
[]
[]
[]
[]
[]
[]
[(490, 1), (713, 1), (1550, 1), (1596, 2)]
[]
[]
[]
[]
[]
[]
[(172, 1), (279, 1), (492, 1), (748, 1), (909, 1), (1058, 1), (1609, 1), (2015, 1), (2116, 1)]
[]
[]
[]
[(279, 1), (575, 1), (2211, 1), (2265, 1)]
[]
[]
[(490, 1), (713, 1), (852, 1)]
[]
[]
[]
[(2177, 1)]
[]
[]
[(490, 1), (713, 1), (1190, 1), (1530, 1), (1799, 1), (2034, 1), (2191, 1)]
[]
[]
[]
[(2174, 1), (2242, 1)]
[]
[]
[(238, 1), (713, 2)]
[]
[]
[]
[(2174, 1)]
[]
[]
[(488, 1), (713, 1), (1392, 1), (2174, 1), (2242, 1), (2321, 1)]
[]
[]
[]
[(2174, 1)]
[]
[]
[(324, 1), (713, 1), (1392, 1), (1990, 1)]
[]
[]
[]
[(2177, 1)]
[]
[]
[(713, 2), (1799, 1), (1900, 1), (2174, 1)]
[]
[]
[]
[(713, 1), (2174, 1)]
[]
[]
[(190, 1), (490, 1), (713, 1), (852, 1)

[]
[]
[]
[]
[]
[(404, 1), (628, 1), (1149, 1), (1723, 1)]
[]
[]
[]
[(394, 1)]
[]
[]
[(1123, 1), (1733, 1), (2174, 1), (2294, 1)]
[]
[]
[]
[(439, 1), (1130, 1)]
[]
[]
[(122, 1), (597, 1)]
[]
[]
[]
[(373, 1)]
[]
[]
[(122, 1), (597, 1)]
[]
[]
[]
[(1201, 1)]
[]
[]
[(122, 1), (597, 1)]
[]
[]
[]
[(2412, 1)]
[]
[]
[(122, 1), (597, 1)]
[]
[]
[]
[]
[]
[]
[(404, 1), (628, 1), (1149, 1), (1723, 1)]
[]
[]
[]
[(394, 1)]
[]
[]
[(447, 1), (615, 1), (658, 1), (953, 1), (1179, 2)]
[]
[]
[]
[(953, 1)]
[]
[]
[(15, 1), (210, 1), (466, 1), (521, 1)]
[]
[]
[]
[]
[]
[]
[(1464, 1), (2321, 1)]
[]
[]
[]
[(1464, 1)]
[]
[]
[(1464, 1), (2321, 1)]
[]
[]
[]
[]
[]
[]
[(1464, 1), (2321, 1)]
[]
[]
[]
[(1464, 1), (2412, 1)]
[]
[]
[(404, 1), (628, 1), (1149, 1), (1723, 1)]
[]
[]
[]
[(394, 1)]
[]
[]
[(1555, 1), (1818, 1), (2321, 1)]
[]
[]
[]
[(235, 1), (992, 1), (1353, 1)]
[]
[]
[(32, 1), (1179, 1), (2210, 1), (2380, 1), (2398, 1)]
[]
[]
[]
[]
[]
[]
[(260, 1), (1402, 2)]
[]
[]
[]
[(2176, 2)]
[]
[]
[(15, 1), (442, 1)]
[]
[

[]
[]
[]
[]
[(421, 1), (827, 1), (1599, 1)]
[]
[]
[]
[(1536, 1)]
[]
[]
[(656, 1), (1273, 1), (2216, 1)]
[]
[]
[]
[(796, 1)]
[]
[]
[(656, 1), (2216, 1)]
[]
[]
[]
[(94, 1)]
[]
[]
[(354, 1), (449, 1), (1879, 1)]
[]
[]
[]
[]
[]
[]
[(434, 1), (550, 1), (1858, 1), (2413, 1)]
[]
[]
[]
[(977, 1), (1905, 1)]
[]
[]
[(197, 1), (656, 1), (1536, 2), (2216, 1)]
[]
[]
[]
[(1536, 1)]
[]
[]
[(58, 1), (251, 1), (434, 1), (1464, 1), (2397, 1)]
[]
[]
[]
[(2412, 1)]
[]
[]
[(238, 1), (497, 1)]
[]
[]
[]
[]
[]
[]
[(385, 1), (713, 1), (1722, 1), (1984, 1)]
[]
[]
[]
[(1353, 1)]
[]
[]
[(296, 1), (488, 1), (491, 1), (950, 1), (1046, 1), (1723, 1), (2230, 1)]
[]
[]
[]
[(1351, 1), (2230, 1)]
[]
[]
[(303, 1), (622, 1), (1003, 1), (1123, 1), (1392, 1), (2145, 1)]
[]
[]
[]
[(210, 1), (981, 1)]
[]
[]
[(462, 1), (713, 1), (1097, 1), (1183, 1), (1484, 1), (1902, 1), (2014, 1)]
[]
[]
[]
[(137, 1), (1530, 1), (2326, 1)]
[]
[]
[(19, 1), (630, 1)]
[]
[]
[]
[(804, 1), (992, 1), (1353, 1), (2312, 1), (2412, 1)]
[]
[]
[(713, 1)

[]
[]
[(210, 2), (216, 1), (488, 1)]
[]
[]
[]
[]
[]
[]
[(35, 1), (1402, 1), (1902, 1)]
[]
[]
[]
[(2176, 1)]
[]
[]
[(210, 1), (238, 1), (308, 1), (1794, 1), (2120, 1), (2235, 1)]
[]
[]
[]
[]
[]
[]
[(713, 1), (722, 1), (1013, 1), (1179, 1), (1464, 1), (1651, 1), (2005, 1), (2130, 1)]
[]
[]
[]
[(1402, 1)]
[]
[]
[(829, 1), (1179, 1), (1197, 1), (2265, 1), (2310, 1)]
[]
[]
[]
[(2265, 1), (2412, 1)]
[]
[]
[(417, 1), (712, 1), (1429, 1), (1561, 1), (2289, 1)]
[]
[]
[]
[]
[]
[]
[(713, 1), (1183, 1), (1484, 1), (1726, 1), (1876, 1), (1900, 1), (1902, 1), (1986, 1)]
[]
[]
[]
[(1806, 1), (1870, 1), (2326, 1)]
[]
[]


In [8]:
from gensim.models import LdaModel

lda_model = LdaModel(corpus, id2word=dictionary, num_topics=10)
print(lda_model)

LdaModel(num_terms=2419, num_topics=10, decay=0.5, chunksize=2000)


In [9]:
def get_topic_term_prob(lda_model):
    topic_term_freqs = lda_model.state.get_lambda()
    topic_term_prob = topic_term_freqs / topic_term_freqs.sum(axis=1)[:, None]
    return topic_term_prob

In [10]:
print(lda_model.alpha.shape) # (n_topics,)
print(lda_model.alpha.sum()) # 1.0

topic_term_prob = get_topic_term_prob(lda_model)
print(topic_term_prob.shape)     # (n_topics, n_terms)
print(topic_term_prob[0].sum())  # 1.0

(10,)
1.0
(10, 2419)
1.0


In [11]:
import pyLDAvis.gensim as gensimvis

prepared_data = gensimvis.prepare(lda_model, corpus, dictionary)

C:\Users\BOGYUNG\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [12]:
pyLDAvis.display(prepared_data)
# pyLDAvis.show(prepared_data)